In [1]:
#Import dependencies
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, text
from sqlalchemy import func

In [2]:
# create engine to smoking_database.sqlite
engine = create_engine("sqlite:///smoking_database.sqlite")

In [3]:
# reflect an existing database into a new model
Base=automap_base()

# reflect the tables
Base.prepare(engine,reflect=True)

In [4]:
# View all of the classes that automap found
Base.classes.keys()

['master']

In [5]:
# Save references to each table
master=Base.classes.master

In [6]:
# Create session (link) from Python to the DB
session=Session(engine)

In [7]:
#Top 10 most prevalent smoking countries in the year 2000
session.query(master.CountryName,master.PercentageTotal).\
filter(master.Year==2000).order_by(master.PercentageTotal.desc()).limit(10).all()
    

[('Kiribati', 35.400002),
 ('Chile', 33.099998),
 ('Denmark', 29.799999),
 ('Greece', 28.799999),
 ('Lebanon', 28.700001),
 ('Bosnia and Herzegovina', 27.799999),
 ('Macedonia', 27.0),
 ('France', 26.9),
 ('Norway', 26.799999),
 ('United Kingdom', 26.0)]

In [8]:
#Top 10 least prevalent smoking countries in the year 2000
session.query(master.CountryName,master.PercentageTotal).\
filter(master.Year==2000).order_by(master.PercentageTotal).limit(10).all()

[('Morocco', 0.60000002),
 ('Oman', 0.60000002),
 ('Algeria', 0.69999999),
 ('Azerbaijan', 0.80000001),
 ('Egypt', 0.80000001),
 ('Eritrea', 0.80000001),
 ('Gambia', 0.80000001),
 ('Ethiopia', 1.0),
 ('Ghana', 1.0),
 ('Lesotho', 1.0)]

In [9]:
#Differences in smoking prevalence between countries in the year 2000 to 2012
yr_2000_total=session.query(master.CountryName,master.PercentageTotal).\
filter(master.Year==2000).all()

yr_2012_total=session.query(master.CountryName,master.PercentageTotal).\
filter(master.Year==2012).all()

In [10]:
#Creating each year Percentage Total into a DatFrame and merging them
total_2000=pd.DataFrame(yr_2000_total)
total_2012=pd.DataFrame(yr_2012_total)


total_diff=pd.merge(total_2000,total_2012,on="CountryName")
total_diff.head()

,CountryName,PercentageTotal_x,PercentageTotal_y
0,Afghanistan,2.3,2.5
1,Albania,3.0,3.9
2,Algeria,0.7,0.9
3,Andorra,24.4,22.0
4,Angola,1.3,1.5


In [11]:
#Creating a % difference column for DataFrame
total_diff['Difference']=total_diff['PercentageTotal_y'].sub(total_diff['PercentageTotal_x'])
total_diff.head()

,CountryName,PercentageTotal_x,PercentageTotal_y,Difference
0,Afghanistan,2.3,2.5,0.2
1,Albania,3.0,3.9,0.9
2,Algeria,0.7,0.9,0.2
3,Andorra,24.4,22.0,-2.4
4,Angola,1.3,1.5,0.2


In [12]:
#Top 10 declining countries in  Total Percentage of smoking prevalence
sort_diff=total_diff.sort_values(by='Difference',ascending=True)
sort_diff.head(10)

,CountryName,PercentageTotal_x,PercentageTotal_y,Difference
46,Denmark,29.799999,17.500000,-12.299999
74,Iceland,23.700001,12.900000,-10.800001
20,Bolivia,21.500000,10.800000,-10.700000
124,Norway,26.799999,16.100000,-10.699999
117,Nepal,23.600000,14.300000,-9.300000
92,Lebanon,28.700001,20.200001,-8.500000
34,Chile,33.099998,25.200001,-7.899997
119,New Zealand,23.900000,16.200001,-7.699999
177,United Kingdom,26.000000,18.700001,-7.299999
161,Switzerland,24.600000,17.400000,-7.200000


In [47]:
#Bottom 10 declining countries in  Total Percentage of smoking prevalence
sort_diff1=total_diff.sort_values(by='Difference',ascending=False)
sort_diff1.head(20)

,CountryName,PercentageTotal_x,PercentageTotal_y,Difference
169,Tonga,18.700001,23.600000,4.899999
136,Romania,14.500000,17.200001,2.700001
134,Portugal,10.800000,13.500000,2.700000
65,Greece,28.799999,31.299999,2.500000
39,Costa Rica,5.300000,7.700000,2.400000
96,Lithuania,11.200000,13.500000,2.300000
171,Tunisia,2.600000,4.500000,1.900000
89,Kyrgyzstan,1.700000,3.300000,1.600000
137,Russia,13.300000,14.900000,1.600000
76,Indonesia,1.900000,3.300000,1.400000


In [14]:
#Top 10 countries with the highest male smoking prevalence in the year 2000
session.query(master.CountryName,master.PercentageMale).\
filter(master.Year==2000).order_by(master.PercentageMale.desc()).limit(10).all()

[('Kiribati', 49.400002),
 ('Greece', 38.900002),
 ('Papua New Guinea', 38.099998),
 ('Macedonia', 37.599998),
 ('Bosnia and Herzegovina', 36.299999),
 ('Chile', 35.5),
 ('Nepal', 35.5),
 ('Lebanon', 35.299999),
 ('Bulgaria', 34.299999),
 ('Bolivia', 33.799999)]

In [15]:
#Top 10 countries with the lowest male smoking prevalence in the year 2000
session.query(master.CountryName,master.PercentageMale).\
filter(master.Year==2000).order_by(master.PercentageMale).limit(10).all()

[('Sao Tome and Principe', 3.0),
 ('Antigua and Barbuda', 3.5999999),
 ('Ethiopia', 3.8),
 ('Ghana', 4.5999999),
 ('Niger', 5.4000001),
 ('Nigeria', 5.4000001),
 ('Sudan', 5.5999999),
 ('Dominica', 5.9000001),
 ('Swaziland', 6.0),
 ('Togo', 6.0)]

In [16]:
#Differences in smoking prevalence between males in countries in the year 2000 to 2012
yr_2000_male=session.query(master.CountryName,master.PercentageMale).\
filter(master.Year==2000).all()

yr_2012_male=session.query(master.CountryName,master.PercentageMale).\
filter(master.Year==2012).all()

In [17]:
#Creating each year Percentage Total into a DatFrame and merging them
male_2000=pd.DataFrame(yr_2000_male)
male_2012=pd.DataFrame(yr_2012_male)


male_diff=pd.merge(male_2000,male_2012,on="CountryName")
male_diff.head()

,CountryName,PercentageMale_x,PercentageMale_y
0,Afghanistan,11.8,11.9
1,Albania,20.0,20.5
2,Algeria,15.8,11.2
3,Andorra,30.5,26.6
4,Angola,8.9,8.7


In [18]:
#Creating a % difference column for DataFrame
male_diff['Difference']=male_diff['PercentageMale_y'].sub(male_diff['PercentageMale_x'])
male_diff.head()

,CountryName,PercentageMale_x,PercentageMale_y,Difference
0,Afghanistan,11.8,11.9,0.1
1,Albania,20.0,20.5,0.5
2,Algeria,15.8,11.2,-4.6
3,Andorra,30.5,26.6,-3.9
4,Angola,8.9,8.7,-0.2


In [19]:
#Top 10 declining countries in Male Percentage of smoking prevalence
sort_mdiff=male_diff.sort_values(by='Difference',ascending=True)
sort_mdiff.head(10)

,CountryName,PercentageMale_x,PercentageMale_y,Difference
46,Denmark,31.400000,18.700001,-12.699999
117,Nepal,35.500000,23.799999,-11.700001
124,Norway,27.700001,16.900000,-10.800001
74,Iceland,24.500000,14.400000,-10.100000
20,Bolivia,33.799999,24.400000,-9.399999
83,Japan,29.799999,21.299999,-8.500000
87,Kiribati,49.400002,41.099998,-8.300004
92,Lebanon,35.299999,27.000000,-8.299999
161,Switzerland,28.000000,19.799999,-8.200001
119,New Zealand,24.200001,16.799999,-7.400002


In [20]:
#Bottom 10 declining countries in Male Percentage of smoking prevalence
sort_mdiff1=male_diff.sort_values(by='Difference',ascending=False)
sort_mdiff1.head(10)

,CountryName,PercentageMale_x,PercentageMale_y,Difference
12,Bahrain,12.500000,19.000000,6.500000
85,Kazakhstan,18.900000,24.400000,5.500000
95,Libya,11.000000,15.400000,4.400000
111,Mongolia,22.500000,25.400000,2.900000
135,Qatar,15.600000,18.100000,2.500000
39,Costa Rica,9.700000,12.100000,2.400000
51,Egypt,16.000000,18.400000,2.400000
76,Indonesia,27.900000,30.200001,2.300001
10,Azerbaijan,20.299999,22.400000,2.100001
143,Saudi Arabia,13.000000,15.000000,2.000000


In [21]:
#Top 10 countries with the highest female smoking prevalence in the year 2000
session.query(master.CountryName,master.PercentageFemale).\
filter(master.Year==2000).order_by(master.PercentageFemale.desc()).limit(10).all()

[('Kiribati', 64.0),
 ('Timor', 57.200001),
 ('South Korea', 56.099998),
 ('Armenia', 55.700001),
 ('Papua New Guinea', 54.5),
 ('Indonesia', 54.299999),
 ('Laos', 53.599998),
 ('Ukraine', 52.700001),
 ('Belarus', 51.200001),
 ('Russia', 50.700001)]

In [22]:
#Top 10 countries with the lowest female smoking prevalence in the year 2000
session.query(master.CountryName,master.PercentageFemale).\
filter(master.Year==2000).order_by(master.PercentageFemale).limit(10).all()

[('Sao Tome and Principe', 5.3000002),
 ('Antigua and Barbuda', 5.5),
 ('Ethiopia', 6.8000002),
 ('Ghana', 8.1000004),
 ('Niger', 8.8000002),
 ('Dominica', 8.8999996),
 ('Nigeria', 9.3000002),
 ('Sudan', 9.6999998),
 ('Cape Verde', 9.8999996),
 ('Suriname', 10.3)]

In [23]:
#Differences in smoking prevalence between countries in the year 2000 to 2012
yr_2000_female=session.query(master.CountryName,master.PercentageFemale).\
filter(master.Year==2000).all()

yr_2012_female=session.query(master.CountryName,master.PercentageFemale).\
filter(master.Year==2012).all()

In [24]:
#Creating each year Percentage Total into a DatFrame and merging them
female_2000=pd.DataFrame(yr_2000_female)
female_2012=pd.DataFrame(yr_2012_female)


female_diff=pd.merge(female_2000,female_2012,on="CountryName")
female_diff.head()

,CountryName,PercentageFemale_x,PercentageFemale_y
0,Afghanistan,21.000000,21.100000
1,Albania,37.599998,37.299999
2,Algeria,30.700001,21.299999
3,Andorra,37.000000,31.500000
4,Angola,16.900000,16.299999


In [25]:
#Creating a % difference column for DataFrame
female_diff['Difference']=female_diff['PercentageFemale_y'].sub(female_diff['PercentageFemale_x'])
female_diff.head()

,CountryName,PercentageFemale_x,PercentageFemale_y,Difference
0,Afghanistan,21.000000,21.100000,0.100000
1,Albania,37.599998,37.299999,-0.299999
2,Algeria,30.700001,21.299999,-9.400002
3,Andorra,37.000000,31.500000,-5.500000
4,Angola,16.900000,16.299999,-0.600001


In [26]:
#Top 10 declining countries in Female Percentage of smoking prevalence
sort_fdiff=female_diff.sort_values(by='Difference',ascending=True)
sort_fdiff.head(10)

,CountryName,PercentageFemale_x,PercentageFemale_y,Difference
83,Japan,48.200001,33.900002,-14.299999
117,Nepal,47.900002,34.500000,-13.400002
154,South Korea,56.099998,42.700001,-13.399997
46,Denmark,33.000000,19.900000,-13.100000
124,Norway,28.700001,17.600000,-11.100001
87,Kiribati,64.000000,53.299999,-10.700001
153,South Africa,31.500000,21.700001,-9.799999
99,Madagascar,35.700001,26.100000,-9.600001
65,Greece,49.400002,39.900002,-9.500000
2,Algeria,30.700001,21.299999,-9.400002


In [27]:
#Bottom 10 declining countries in Female Percentage of smoking prevalence
sort_fdiff1=female_diff.sort_values(by='Difference',ascending=False)
sort_fdiff1.head(10)

,CountryName,PercentageFemale_x,PercentageFemale_y,Difference
85,Kazakhstan,31.200001,44.500000,13.299999
95,Libya,20.000000,29.900000,9.900000
12,Bahrain,18.000000,26.400000,8.400000
102,Maldives,29.200001,34.400002,5.200001
111,Mongolia,40.500000,45.599998,5.099998
40,Cote d'Ivoire,14.100000,18.799999,4.699999
51,Egypt,31.200001,35.799999,4.599998
10,Azerbaijan,41.599998,45.200001,3.600003
41,Croatia,32.400002,36.000000,3.599998
76,Indonesia,54.299999,57.299999,3.000000


In [38]:
#Top 10 countries with the highest GDP per capita for the year 2012
session.query(master.CountryName,master.PercentageTotal,master.GDPPerCapita).\
filter(master.Year==2012,master.GDPPerCapita != '').order_by(master.GDPPerCapita.desc()).limit(30).all()


[('Luxembourg', 21.4, 112584.6763),
 ('Norway', 16.1, 102175.9193),
 ('Qatar', 1.4, 98041.36224),
 ('Switzerland', 17.4, 85836.20768),
 ('Australia', 14.7, 68047.37818),
 ('Denmark', 17.5, 58507.50805),
 ('Sweden', 14.7, 58037.82132),
 ('Singapore', 3.8, 55547.54625),
 ('Canada', 12.6, 52669.08996),
 ('United States', 13.7, 51784.41857),
 ('Kuwait', 3.0999999, 51271.14865),
 ('Netherlands', 20.1, 50070.1416),
 ('Japan', 9.5, 49145.28043),
 ('Ireland', 23.799999, 49054.69743),
 ('Austria', 24.4, 48564.91734),
 ('Finland', 13.5, 47708.06128),
 ('Iceland', 12.9, 45995.54788),
 ('Andorra', 22.0, 44902.38077),
 ('Belgium', 23.4, 44670.56068),
 ('United Arab Emirates', 1.7, 44386.78608),
 ('Germany', 18.9, 43855.85447),
 ('United Kingdom', 18.700001, 42485.58607),
 ('France', 23.700001, 40870.85236),
 ('New Zealand', 16.200001, 39973.38076),
 ('Italy', 19.299999, 35051.52127),
 ('Israel', 13.4, 33156.22832),
 ('Cyprus', 17.5, 28991.92969),
 ('Spain', 20.6, 28322.94659),
 ('Bahrain', 5.300000

In [42]:
#Bottom 10 countries with the lowest GDP per capita for the year 2012
session.query(master.CountryName,master.PercentageTotal,master.GDPPerCapita).\
filter(master.Year==2012,master.GDPPerCapita != '').order_by(master.GDPPerCapita).limit(20).all()


[('Burundi', 8.1000004, 238.2059453),
 ('Ethiopia', 0.80000001, 458.5509208),
 ('Madagascar', 1.4, 504.1737378),
 ('Niger', 1.3, 525.0472844),
 ('Central African Republic', 1.4, 525.8675036),
 ('Sierra Leone', 6.0, 558.1797747),
 ('Malawi', 1.8, 563.0615397),
 ('Guinea-Bissau', 1.6, 598.5726262),
 ('Liberia', 1.1, 644.4555767),
 ('Afghanistan', 2.5, 663.1410528),
 ('Mozambique', 3.4000001, 667.7177895),
 ('Rwanda', 1.9, 706.139034),
 ('Guinea', 1.4, 707.9676782),
 ('Burkina Faso', 2.5, 733.9728789),
 ('Mali', 2.9000001, 753.3921368),
 ('Togo', 1.2, 759.8959722),
 ('Nepal', 14.3, 794.0925593),
 ('Uganda', 1.5, 796.7111393),
 ('Tanzania', 1.1, 854.5432129),
 ('Bangladesh', 1.4, 876.8180068)]

In [43]:
#Top 10 countries with the highest Health expenditure for the year 2012
session.query(master.CountryName,master.PercentageTotal,master.HealthExpenditure).\
filter(master.Year==2012,master.HealthExpenditure != '').order_by(master.HealthExpenditure.desc()).limit(20).all()


[('Norway', 16.1, 8970.120117),
 ('Switzerland', 17.4, 8917.119141),
 ('United States', 13.7, 8272.958008),
 ('Sweden', 14.7, 6254.442383),
 ('Denmark', 17.5, 6168.339355),
 ('Australia', 14.7, 6071.650879),
 ('Luxembourg', 21.4, 6029.475098),
 ('Canada', 12.6, 5541.577148),
 ('Netherlands', 20.1, 5284.649902),
 ('Japan', 9.5, 5235.416016),
 ('Ireland', 23.799999, 5222.044922),
 ('Austria', 24.4, 4966.095703),
 ('Germany', 18.9, 4765.694824),
 ('Belgium', 23.4, 4694.45752),
 ('France', 23.700001, 4644.595703),
 ('Finland', 13.5, 4573.292969),
 ('United Kingdom', 18.700001, 4253.501953),
 ('New Zealand', 16.200001, 3856.173096),
 ('Iceland', 12.9, 3759.893555),
 ('Italy', 19.299999, 3086.303223)]

In [45]:
#Top 10 countries with the lowest Health expenditure for the year 2012
session.query(master.CountryName,master.PercentageTotal,master.HealthExpenditure).\
filter(master.Year==2012,master.HealthExpenditure != '').order_by(master.HealthExpenditure).limit(20).all()


[('Madagascar', 1.4, 19.27203178),
 ('Ethiopia', 0.80000001, 20.28753662),
 ('Burundi', 8.1000004, 20.31077957),
 ('Central African Republic', 1.4, 20.60425186),
 ('Niger', 1.3, 22.56604767),
 ('Bangladesh', 1.4, 23.89686203),
 ('Guinea', 1.4, 23.99205208),
 ('Eritrea', 0.60000002, 26.20994949),
 ('Pakistan', 4.5999999, 26.28877595),
 ('Mali', 2.9000001, 28.26860619),
 ('Malawi', 1.8, 32.80560684),
 ('Burkina Faso', 2.5, 33.78236771),
 ('Nepal', 14.3, 33.90107346),
 ('Mozambique', 3.4000001, 35.03115845),
 ('Togo', 1.2, 36.26974869),
 ('Myanmar', 5.9000001, 36.97321701),
 ('Chad', 1.9, 37.49700546),
 ('Guinea-Bissau', 1.6, 37.72527695),
 ('Benin', 1.6, 37.98866653),
 ('Tanzania', 1.1, 41.63511276)]

In [46]:
#Top 10 countries with the highest rate of unemployment for the year 2012
session.query(master.CountryName,master.PercentageTotal,master.Unemployment).\
filter(master.Year==2012,master.Unemployment != '').order_by(master.Unemployment.desc()).limit(30).all()


[('Bosnia and Herzegovina', 23.299999, 28.01),
 ('Djibouti', 6.4000001, 26.873),
 ('Spain', 20.6, 24.79),
 ('Greece', 31.299999, 24.44),
 ('Serbia', 21.5, 24.0),
 ('South Africa', 8.6999998, 21.79),
 ('Gabon', 2.9000001, 20.489),
 ('Montenegro', 19.1, 19.81),
 ('Georgia', 4.1999998, 19.65),
 ('Somalia', 1.8, 19.161),
 ('Libya', 0.89999998, 19.03),
 ('Botswana', 5.1999998, 18.416),
 ('Sudan', 1.5, 17.668),
 ('Tunisia', 4.5, 17.63),
 ('Lesotho', 0.80000001, 16.813),
 ('Namibia', 8.3000002, 16.77),
 ('Croatia', 20.6, 15.93),
 ('Portugal', 13.5, 15.53),
 ('Ireland', 23.799999, 15.45),
 ('Latvia', 16.5, 15.05),
 ('Haiti', 2.9000001, 14.1),
 ('Sao Tome and Principe', 1.0, 13.59),
 ('Albania', 3.9000001, 13.38),
 ('Lithuania', 13.5, 13.36),
 ('Guyana', 3.5, 12.333),
 ('Bulgaria', 26.0, 12.27),
 ('Jordan', 8.1999998, 12.2),
 ('Rwanda', 1.9, 11.858),
 ('Cyprus', 17.5, 11.8),
 ('Barbados', 1.9, 11.59)]

In [33]:
#Top 10 countries with the lowest rate of unemployment for the year 2012
session.query(master.CountryName,master.PercentageTotal,master.Unemployment).\
filter(master.Year==2012,master.Unemployment != '').order_by(master.Unemployment).limit(10).all()


[('Qatar', 1.4, 0.48),
 ('Cambodia', 3.5, 0.509),
 ('Thailand', 2.4000001, 0.58),
 ('Madagascar', 1.4, 0.6),
 ('Myanmar', 5.9000001, 0.731),
 ('Chad', 1.9, 1.0),
 ('Vietnam', 1.5, 1.03),
 ('Solomon Islands', 15.3, 1.055),
 ('Bahrain', 5.3000002, 1.2),
 ('Mali', 2.9000001, 1.385)]

In [34]:
#Smoking prevalence worldwide-2000 
session.query(master.Year, func.sum(master.PercentageTotal/188)).\
filter(master.Year==2000).all()

[(2000, 9.091489344521275)]

In [35]:
#Smoking prevalence worldwide-2012 
session.query(master.Year, func.sum(master.PercentageTotal/188)).\
filter(master.Year==2012).all()

[(2012, 7.933510641276595)]